Packages

In [1]:
import numpy as np
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

Dataset

In [2]:
import h5py
with h5py.File('usps.h5', 'r') as hf:
    train = hf.get('train')
    X_tr = train.get('data')[:]
    y_tr = train.get('target')[:]

X = X_tr
X = X.astype('float')
# StanScal = StandardScaler()
# X = StanScal.fit_transform(X)
feature_mean = np.mean(X, axis=0)
X = X - feature_mean
X = X.T
d = X.shape[0]
N = X.shape[1]
print(d)
print(N)

256
7291


Functions

In [3]:
def norm(x):
    return np.sqrt(np.mean(x**2))

Number of Selected Features

In [4]:
eps = 0.03

Selected Features

In [5]:
d_j = X.copy()

j_ind = int(np.argmax(np.var(d_j,axis=1)))
j_max = np.array([j_ind])
z = np.array([X[j_ind,:]])
d_j[j_ind] = 0
z1_hat = z[0]/norm(z[0])
F_hat = np.array([z1_hat])

eigenvals = np.array(np.max(np.var(X,axis=1)))

d_j = d_j - np.matmul(np.array([np.mean(d_j*F_hat[0],axis=1)]).T, np.array([F_hat[0]]))

j_ind = int(np.argmax(np.var(d_j,axis=1)))
j_max = np.append(j_max, j_ind)
z = np.append(z, np.array([X[j_ind,:]]), axis=0)
d_j[j_ind] = 0
z2_hat = z[1]/norm(z[1])
F_hat = np.append(F_hat, np.array([z2_hat]), axis=0)
H_1 = np.array([z[0], z[1]])
z12 = z[0]*z[1] - np.mean(z[0]*z[1])
H_2 = np.array([z12])
H_2_j = H_2
z12_tilde = z12
for i in range(len(F_hat)):
    z12_tilde = z12_tilde - np.mean(z12*F_hat[i])*F_hat[i]
z12_hat = z12_tilde/norm(z12_tilde)
F_hat = np.append(F_hat, np.array([z12_hat]), axis=0)
F_j_hat = np.array([z12_hat])

for i in range(len(F_hat)):
    d_j = d_j - np.matmul(np.array([np.mean(d_j*F_hat[i],axis=1)]).T, np.array([F_hat[i]]))

eigenval_j = np.max(np.var(d_j,axis=1))
eigenvals = np.append(eigenvals, eigenval_j)
counter = 2

In [6]:
while eigenval_j>eps:
    j_ind = int(np.argmax(np.var(d_j,axis=1)))
    j_max = np.append(j_max, j_ind)
    z = np.append(z, np.array([X[j_ind,:]]), axis=0)
    d_j[j_ind] = 0
    z_j = X[j_ind,:]
    z_j_tilde = z_j
    for F_hat_ind in range(len(F_hat)):
        z_j_tilde = z_j_tilde - np.mean(z_j*F_hat[F_hat_ind])*F_hat[F_hat_ind]
    z_j_hat = z_j_tilde/norm(z_j_tilde)
    F_hat = np.append(F_hat, np.array([z_j_hat]), axis=0)
    F_j_hat = np.array([z_j_hat])
    H_2_j = np.empty((len(H_1), N))
    H_2_j[:] = np.nan
    for H_1_ind in range(len(H_1)):
        z_j_2 = z_j*H_1[H_1_ind] - np.mean(z_j*H_1[H_1_ind])
        H_2_j[H_1_ind] = z_j_2
        z_j_2_tilde = z_j_2
        for F_hat_ind in range(len(F_hat)):
            z_j_2_tilde = z_j_2_tilde - np.mean(z_j_2*F_hat[F_hat_ind])*F_hat[F_hat_ind]
        z_j_2_hat = z_j_2_tilde/norm(z_j_2_tilde)
        F_hat = np.append(F_hat, np.array([z_j_2_hat]), axis=0)
        F_j_hat = np.append(F_j_hat, np.array([z_j_2_hat]), axis=0)
    H_1 = np.append(H_1, np.array([z_j]), axis=0)

    H_3_j = np.empty((len(H_2), N))
    H_3_j[:] = np.nan
    for H_2_ind in range(len(H_2)):
        z_j_3 = z_j*H_2[H_2_ind]
        H_3_j[H_2_ind] = z_j_3
        z_j_3_tilde = z_j_3
        for F_hat_ind in range(len(F_hat)):
            z_j_3_tilde = z_j_3_tilde - np.mean(z_j_3*F_hat[F_hat_ind])*F_hat[F_hat_ind]
        z_j_3_hat = z_j_3_tilde/norm(z_j_3_tilde)
        F_hat = np.append(F_hat, np.array([z_j_3_hat]), axis=0)
        F_j_hat = np.append(F_j_hat, np.array([z_j_3_hat]), axis=0)
    H_2 = np.append(H_2, H_2_j, axis=0)

    for i in range(len(F_j_hat)):
        d_j = d_j - np.matmul(np.array([np.mean(d_j*F_j_hat[i],axis=1)]).T, np.array([F_j_hat[i]]))

    eigenval_j = np.max(np.var(d_j,axis=1))
    eigenvals = np.append(eigenvals, eigenval_j)
    print(counter, ':', eigenval_j)
    counter = counter + 1
    # if counter>50:
    #     break

2 : 0.15466043018617723
3 : 0.146522183181084
4 : 0.13903951113742516
5 : 0.13055088207144017
6 : 0.12717084013486302
7 : 0.11229660264525419
8 : 0.10277842020332383
9 : 0.09423839989706612
10 : 0.09014832553950028
11 : 0.08559786879961061
12 : 0.07911536091351863
13 : 0.07596677669316156
14 : 0.07121989719548458
15 : 0.0641008880937113
16 : 0.059230338364953145
17 : 0.05051778249049964
18 : 0.04674092851004701
19 : 0.04190499849087088
20 : 0.03870164067691324
21 : 0.034359268025073185
22 : 0.03273625173833076
23 : 0.03086486403048717
24 : 0.030124177838121657
25 : 0.028428210385932712


In [7]:
j_max.shape

(26,)

In [8]:
j_max

array([229,  55,  68, 105,  26, 201, 134, 107, 153, 101,  59,  38, 197,
         8, 171,  51,  57, 248, 179, 214, 151, 115,  88,  21, 203, 139])

In [9]:
del train, X_tr, y_tr, h5py, hf
import dill
dill.dump_session('GFS-USPS-2.pkl')